# Limpando os dados 

### Importando a base de clientes de um banco com o resultado do pagamento ou não de um empréstimo

In [52]:
#Importando o pandas
import pandas as pd

In [53]:
# Importando a base de dados
base = pd.read_excel("ChavesClientes.xlsx",sheet_name = "base")

In [54]:
# Visualizando a base de dados
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0


**Verificando a cardinalidade desses dados**

In [55]:
# Verficando as informações da base
base.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 932.0+ bytes


In [56]:
# Verificar a cardidade da dos dados
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

### Ao analisar essa base, percebemos que existe uma alta cardinalidade e por isso precisamos tratar esses dados

- Ao mapear com o responsável pela área, ele informou o que representa cada coluna:
    - ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
    - ClassRisco: formado por:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa
    - CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
        - Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta

**Podemos começar tratando simplesmente separando a string da coluna**

In [57]:
# Separando a coluna ChaveSituacao tratamndo a idade e o sexo
texto = '32FC'

In [58]:
# separando a idade da coluna ChaveSituacao
texto[:2]

'32'

In [59]:
# SEparando o sexo da coluna ChaveSituacao  
texto[2:3]

'F'

In [60]:
# Separando a ultima letra da coluna ChaveSituacao
texto[-1]

'C'

In [61]:
# Criando as novas colunas
base['Idade'] = base.ChaveSituacao.str[:2]
base['Genero'] = base.ChaveSituacao.str[2:3]
base['EstadoCivil'] = base.ChaveSituacao.str[-1]

In [62]:
# Visualizando as novas colunas
base.head()
display(base )

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D
5,6,28FC,C-Amarelo,Platinum-Alpha,0,28,F,C
6,7,27MD,A-Verde,Platinum-Beta,1,27,M,D
7,8,31MD,C-Cinza,Basic,0,31,M,D
8,9,28FS,A-Cinza,Black,1,28,F,S
9,10,31MV,C+Amarelo,Platinum,1,31,M,V


In [63]:
# Verificar a cardidade da dos dados
# Visualizando dados da base de dados na coluna Pagamento e EstadoCivil 
base.groupby(["Pagamento","EstadoCivil"])["Pagamento"].count()

Pagamento  EstadoCivil
0          C              3
           D              4
1          C              4
           D              3
           S              1
           V              5
Name: Pagamento, dtype: int64

##### obs: 
###### Neste caso temos menos cardinalidade e menos dados .Interessante notar que 5 viuvos pagaram . 
###### Sendo assim para essa empresa focar neste tipo de cliente pode ser vantajoso 

**Podemos também fazer o split (divisão) de um valor baseado em um delimitador**

In [64]:
# Visualizando a base de dados
display(base)
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D
5,6,28FC,C-Amarelo,Platinum-Alpha,0,28,F,C
6,7,27MD,A-Verde,Platinum-Beta,1,27,M,D
7,8,31MD,C-Cinza,Basic,0,31,M,D
8,9,28FS,A-Cinza,Black,1,28,F,S
9,10,31MV,C+Amarelo,Platinum,1,31,M,V


,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


In [65]:
# Separando a coluna CatCliente tratamndo a categoria e o VIP 
texto = 'Basic-Alpha'

In [66]:
# Separando a Categoria da coluna CatCliente 
texto.split('-')

['Basic', 'Alpha']

In [67]:
#  Criando as novas colunas Categoria e CatVIP tratamndo a categoria   
base['Categoria'] = base.CatCliente.str.split('-').str.get(0)
base['CatVIP'] = base.CatCliente.str.split('-').str.get(1)

In [68]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


**Caso a gente não consiga separar pela posição do texto ou por um delimitador, ainda temos a opção de usar o Regex**

In [69]:
import re

In [70]:
re.findall('Lucas','meu nome é Lucas')

['Lucas']

- Regex:
    - ^: Começa com
    - $: Termina com
    - *: O último caracter repetido 0 ou mais vezes
    - +: O último caracter repetido 1 ou mais vezes
    - ?: O último caracter repetido 0 ou 1 vez
    - [A-Z]: qualquer valor em maiúsculo

In [71]:
# Buscando Lucas no texto abaixo
re.findall('Lucas','meu nome é Lucas')

['Lucas']

In [72]:
# Verificando se começa ou termina com um caracter (usando ˆ, $)
re.findall('^meu','meu nome é Lucas')

['meu']

In [73]:
# Verificando se existe parte da palavra Lucas (usando *, +, ?)
re.findall('Lu?','meu nome é Lucas')

['Lu']

In [74]:
# Buscando por letras maiúsculas
re.findall('[A-Z]','meu nome é Lcas')

['L']

In [75]:
# Buscando agora a classificação do cliente dentro da coluna "ClassRisco"
re.findall("^[A-Z][^A-Z]?","B+amarelo")

['B+']

In [76]:
base["Risco"] = base.ClassRisco.apply(lambda x: re.findall("^[A-Z][^A-Za-z]?",x)[0])

In [77]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-


In [78]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


**Transformando a coluna 'Idade' em numérico**

In [79]:
base['Idade'] = pd.to_numeric(base.Idade)

In [80]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


**Por fim, tratando os valores vazios**

In [81]:
base.loc[base.CatVIP.isnull(),"CatVIP"] = "Comum"

In [82]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         20 non-null     object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


**Agora podemos criar uma nova base apenas com as colunas que acabamos de criar**

In [83]:
base = base[["Idade","Genero","EstadoCivil","Categoria","CatVIP","Risco","Pagamento"]]

In [84]:
display(base)

,Idade,Genero,EstadoCivil,Categoria,CatVIP,Risco,Pagamento
0,32,F,C,Basic,Alpha,C,1
1,25,M,V,Black,Comum,A,1
2,27,M,V,Basic,Beta,B-,1
3,26,F,D,Black,Comum,B,0
4,26,F,D,Black,Comum,C-,0
5,28,F,C,Platinum,Alpha,C-,0
6,27,M,D,Platinum,Beta,A-,1
7,31,M,D,Basic,Comum,C-,0
8,28,F,S,Black,Comum,A-,1
9,31,M,V,Platinum,Comum,C+,1


In [85]:
#  Verificar a cardidade da dos dados categorias e pagamentos
base.groupby(["Pagamento","Categoria"])["Pagamento"].count()

Pagamento  Categoria
0          Basic        3
           Black        3
           Platinum     1
1          Basic        3
           Black        4
           Platinum     6
Name: Pagamento, dtype: int64

##### Obs:
###### Podemos observar que o cliente Platinum é mais assíduo  
###### Os Clientes Black estão no memso nivel

In [86]:
# Verificar a cardidade CatVIP e pagamentos
base.groupby(["Pagamento","CatVIP"])["Pagamento"].count()

Pagamento  CatVIP
0          Alpha     1
           Beta      2
           Comum     4
1          Alpha     3
           Beta      2
           Comum     8
Name: Pagamento, dtype: int64

##### Obs:
###### Podemos observar que o cliente Alpha  cumpre com seu pagamento 
###### Os Clientes Comum esta mantendo um bom numero 
 
 

In [87]:
# Verificar a cardidade Risco e pagamentos 
base.groupby(["Pagamento","Risco"])["Pagamento"].count()

Pagamento  Risco
0          B        2
           C-       5
1          A        3
           A-       4
           B-       2
           C        3
           C+       1
Name: Pagamento, dtype: int64

##### Obs:
###### Podemos observar que o cliente A  cumpre com seu pagamento 
###### Os Clientes C- enenhum pagouo